In [1]:
#importeren relevanten bibliotheken
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
#voorbeeld laten zien in werkgroep 

#Stap 1: Laad en filter de relevante datasets
data_raw = pd.read_csv('migranten_eu.csv')
data_raw.drop(data_raw.loc[data_raw['Year']==1999].index, inplace=True)
data_raw.head(5) 

#Stap 2: xxx

#Stap 3: xxx

#of.....

#Stap 1: Laad de dataset van migranten in EU
data_raw = pd.read_csv('migranten_eu.csv')

#Stap 2: Verwijder de data voor 1999
data_raw.drop(data_raw.loc[data_raw['Year']==1999].index, inplace=True)

#Stap 3: Check of de data is verwijderd
data_raw.head(5) 


#of.....

#Eerst hebben we x gedaan toen hebben y gedaan daarna konden we onrelevante dingen verwijderen. 

data_raw = pd.read_csv('migranten_eu.csv')
data_raw.drop(data_raw.loc[data_raw['Year']==1999].index, inplace=True)
data_raw.head(5)

#Ben in de voorbeeld file bezig alles mooi erin te zetten maar dit is puur even rough laten zien wat 
#de opties zijn en welke de bedoeling is. 

,Unnamed: 0,Year,Country of origin,Country of origin (ISO),Country of asylum,Country of asylum (ISO),Refugees under UNHCR's mandate,Asylum-seekers,IDPs of concern to UNHCR,Other people in need of international protection,Stateless persons,Host Community,Others of concern
1036,1966,2000,Romania,ROU,Germany,DEU,0,32.0,0.0,-,0.0,-,0.0
1037,1967,2000,Romania,ROU,Hungary,HUN,1318,15.0,0.0,-,0.0,-,0.0
1038,1969,2000,Romania,ROU,Ireland,IRL,8,0.0,0.0,-,0.0,-,0.0
1039,1970,2000,Romania,ROU,Italy,ITA,84,0.0,0.0,-,0.0,-,0.0
1040,1971,2000,Romania,ROU,Netherlands (Kingdom of the),NLD,228,40.0,0.0,-,0.0,-,0.0


In [3]:
#Jelle

#Stap 1: Laad en filter de relevante datasets
data_raw = pd.read_csv('migranten_eu.csv')
data_raw.drop(data_raw.loc[data_raw['Year']==1999].index, inplace=True)
data_raw.head(5) 

#Stap 2: migranten totaal europa
migranten_totaal_europa = data_raw[['Year', 'Country of asylum', "Refugees under UNHCR's mandate"]]

migranten_totaal_europa['Country of asylum'] = migranten_totaal_europa['Country of asylum'].replace({'Netherlands (Kingdom of the)': 'Netherlands'})
migranten_totaal_europa['Country of asylum'] = migranten_totaal_europa['Country of asylum'].replace({'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom'})
migranten_totaal_europa['Country of asylum'] = migranten_totaal_europa['Country of asylum'].replace({'Serbia and Kosovo: S/RES/1244 (1999)': 'Serbia'})

migranten_totaal_europa.head(5) 

#sum migranten totaal europa gegroept door jaar en land
df2 = migranten_totaal_europa.groupby(['Year','Country of asylum'], as_index=False)["Refugees under UNHCR's mandate"].sum()
print(df2)
#df2.to_csv('migranten_per_jaar_per_land.csv')

#sum migranten totaal europa gegroept door jaar
df3 = migranten_totaal_europa.groupby(['Year'], as_index=False)["Refugees under UNHCR's mandate"].sum()
print(df3)
#df3.to_csv('migranten_per_jaar_europa.csv')

#sum migranten totaal europa gegroept door land
df4 = migranten_totaal_europa.groupby(['Country of asylum'], as_index=False)["Refugees under UNHCR's mandate"].sum()
print(df4)
#df4.to_csv('migranten_per_land_totaal.csv')

#longtitude en latitude landen
data_lon_lat = pd.read_csv('world_country_and_usa_states_latitude_and_longitude_values.csv').rename(columns={'country': 'Country of asylum'})
data_lon_lat['Country of asylum'] = data_lon_lat['Country of asylum'].replace({'Macedonia [FYROM]': 'North Macedonia'})
data_lon_lat2 = data_lon_lat[['latitude', 'longitude', 'Country of asylum']]
data_lon_lat2.head(5)

#data terrorime en land van asiel
data_ter = pd.read_csv('terrorism_eu.csv').rename(columns={'country_txt': 'Country of asylum', 'iyear': 'Year', 'multiple': 'Terrorist attacks'})
data_ter['Country of asylum'] = data_ter['Country of asylum'].replace({'Macedonia': 'North Macedonia'})
data_ter['Country of asylum'] = data_ter['Country of asylum'].replace({'Slovak Republic': 'Slovakia'})
data_ter['Country of asylum'] = data_ter['Country of asylum'].replace({'Serbia-Montenegro': 'Serbia'})
data_ter = data_ter[['Country of asylum', 'Year', 'Terrorist attacks']]
data_ter.head(5)

#mergede data van df
df_half = df2.merge(data_lon_lat2, on='Country of asylum')
df_full = pd.merge(df_half, data_ter,  how='left', left_on=['Country of asylum','Year'], right_on = ['Country of asylum','Year']).fillna(0)
df_full.head(500) 
#df_full.to_csv('test.csv')

#map figuur 
fig = px.scatter_geo(df_full, 
                     lat='latitude',
                     lon='longitude',
                     color="Refugees under UNHCR's mandate",
                     color_continuous_scale=px.colors.sequential.OrRd,
                     height=650,
                     width=950,
                     hover_name="Country of asylum",
                     hover_data={
                                "Refugees under UNHCR's mandate":True, 
                                "Terrorist attacks":True,
                                "latitude":False, 
                                "longitude":False,
                                },
                     projection="natural earth",
                     scope="europe",
                     size=df_full["Year"],
                     size_max=10,
                     animation_frame="Year",
                     range_color=(0,200000),
                    )


fig.update_geos(
    visible=True, 
    resolution=50,
    showcountries=True, 
    countrycolor="whitesmoke",
    bgcolor='whitesmoke',
    landcolor='#26a783',
)

fig.update_layout(
    title='Hoeveelheid vluchtelingen en terroristische aanvallen tussen 2000 en 2017',
)

fig.update_traces(marker=dict(line=dict(width=0.5, color='black')))
fig.show()


/var/folders/d5/l26d_rb125nf0gsrglrqqnsw0000gn/T/ipykernel_88605/3193170447.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/d5/l26d_rb125nf0gsrglrqqnsw0000gn/T/ipykernel_88605/3193170447.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/d5/l26d_rb125nf0gsrglrqqnsw0000gn/T/ipykernel_88605/3193170447.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

     Year Country of asylum  Refugees under UNHCR's mandate
0    2000           Albania                             521
1    2000           Austria                           15493
2    2000           Belarus                             453
3    2000           Belgium                           18821
4    2000          Bulgaria                            1470
..    ...               ...                             ...
607  2017             Spain                           17541
608  2017            Sweden                          240954
609  2017       Switzerland                           93030
610  2017           Ukraine                            3241
611  2017    United Kingdom                          121821

[612 rows x 3 columns]
    Year  Refugees under UNHCR's mandate
0   2000                         2310144
1   2001                         2260823
2   2002                         2292404
3   2003                         2175223
4   2004                         2053058
5   2005  

In [4]:
#Sally
df = pd.read_csv('globalterrorismdb_0718dist.csv', encoding='latin-1')
df.head()

df1 = df.groupby(['iyear', 'region_txt', 'country_txt']).count().reset_index()
df1

mask = df1[df1['region_txt'].isin(['Eastern Europe', 'Western Europe'])]
mask

df2 = mask[mask.iyear.between(2000, 2017)]
df2

df3 = df2.groupby(['iyear']).count().reset_index()

df3 = df2.groupby('iyear')['eventid'].sum().reset_index().rename(columns = {'iyear' : 'year', 'eventid' : 'count'})

df4 = df2.groupby('country_txt')['eventid'].sum().reset_index().rename(columns = {'country_txt' : 'country', 'eventid' : 'count'})
df4

df3

df5 = df2[['iyear', 'country_txt', 'eventid']].rename(columns = {'iyear' : 'year', 'country_txt' : 'country', 'eventid' : 'count'})
df5

# Totaal aantal aanslagen per jaar in Europa. 
df3
#df3.to_csv('aanslagen_per_jaar.csv', index=False, header=True)

# Totaal aantal aanslagen tussen 2000 en 2017 in Europa.
sum(df3['count'])

# Aantal aanslagen per land per jaar in Europa.
df5
#df6.to_csv('aanslagen_per_jaar_per_land.csv', index=False, header=True)

# Totaal aantal aanslagen per land tussen 2000 en 2017 in Europa.
df4
#df4.to_csv('totale_aanslagen_per_land.csv', index=False, header=True)

FileNotFoundError: [Errno 2] No such file or directory: 'globalterrorismdb_0718dist.csv'